# Face Mask Detection Using YOLOV5

Tutorial: https://towardsdatascience.com/the-practical-guide-for-object-detection-with-yolov5-algorithm-74c04aac4843

Dataset: https://www.kaggle.com/datasets/andrewmvd/face-mask-detection

This notebook is a simple guide to training a face mask detector using YOLOV5. As the dataset used is small, we do not train the model from scratch, but apply transfer learning instead. After training the model, it can be exported to various formats for convenience. 

To train a face mask detector:
1. Download the dataset from the Kaggle link above into your project folder.
2. Run the `createLabel.py` script to convert the annotations into the correct format.
3. Split the images and labels into train-validation-test with your desired ratio. Follow the directory format shown in the tutorial under the Data Handling section.
4. Run the code below.

To make predictions using a pretrained model:
1. In the last cell of this notebook, change the `--source` argument to the directory containing the images (relative to the directory of this notebook).
2. Execute the last cell. Results are saved to the `runs_mask/detect_test` folder.

Potential issues:
1. Out of memory when training model
Fix: If you are training locally, try reducing batch size (argument `--batch-size`, default 16) or number of workers (argument `--workers`, default 8). If problem persists, train model on Google Colab instead. Remember to allocate GPU before training to speed up training.
2. Issue with wandb
Fix: If authentication is required, login to wandb to request for a key. If it is some other error, it might be a network issue, so check your connection or try running the cell again until it works.

Note:
1. The YOLOv5 github repository has been included with this project in the `yolov5` folder. Hence, there is no need to clone the repository.
2. Changes made to the original YOLOv5 repository include:
    1. Adding `mask.yaml` in the `data` folder.
    2. Changing the `nc` parameter in `yolov5s.yaml` to 3, since there are 3 classes of labels.

In [1]:
# !git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 13162, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 13162 (delta 23), reused 28 (delta 13), pack-reused 13119
Receiving objects: 100% (13162/13162), 12.52 MiB | 24.20 MiB/s, done.
Resolving deltas: 100% (9035/9035), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 4.8 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Feature Extraction

In [2]:
!python train.py --batch 32 --epochs 150 --data 'data/mask.yaml' --weights 'yolov5s6.pt' --cache --freeze 12 --project 'runs_mask' --name 'feature_extraction'

train: weights=yolov5s6.pt, cfg=, data=data/mask.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=150, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs_mask, name=feature_extraction, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[12], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (Docker image), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v6.1-325-g3e85863 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, f

## Fine Tuning

- Re-train model on our data
- Note: `hyp.finetune.yaml` (in the tutorial) has been renamed to `hyp.VOC.yaml` (source: https://github.com/ultralytics/yolov5/issues/6820)

In [4]:
!python train.py --hyp 'hyp.VOC.yaml' --batch 16 --epochs 100 --data 'data/mask.yaml' --weights 'runs_mask/feature_extraction/weights/best.pt' --project 'runs_mask' --name 'fine-tuning' --cache

train: weights=runs_mask/feature_extraction/weights/best.pt, cfg=, data=data/mask.yaml, hyp=hyp.VOC.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs_mask, name=fine-tuning, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (Docker image), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v6.1-325-g3e85863 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.00334, lrf=0.15135, momentum=0.74832, weight_decay=0.00025, warmup_epochs=3.3835, warmup_momentum=0.59462, warmup_bias_lr=0.18657, box=0.02, cls=0.21638, cls_pw=0.5, obj=0.51728, obj

## Validation

- Using trained model stored in `runs_mask/fine-tuning/weights/best.pt`.
- Results are stored in `runs_mask/Validation`.

In [5]:
!python val.py --batch 64 --data 'data/mask.yaml' --weights 'runs_mask/fine-tuning/weights/best.pt' --task test --project 'runs_mask' --name 'Validation' --augment

val: data=data/mask.yaml, weights=['runs_mask/fine-tuning/weights/best.pt'], batch_size=64, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs_mask, name=Validation, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-325-g3e85863 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 280 layers, 12315904 parameters, 0 gradients, 16.1 GFLOPs
test: Scanning '/content/yolov5/../datasets/mask/labels/test' images and labels...85 found, 0 missing, 0 empty, 0 corrupt: 100% 85/85 [00:00<00:00, 146.06it/s]
test: New cache created: /content/yolov5/../datasets/mask/labels/test.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 2/2 [00:05<00:00,  2.76s/it]
                 all         85        370      0.917      0.695        0.8      0.505
        without ma

## Inference / Testing Using Trained Model

- Using trained model stored at `runs_mask/fine-tuning/weights/best.pt`.
- Images with predicted bounding boxes and labels are saved to `runs_mask/detect_test` folder.

In [6]:
!python detect.py --source '../datasets/mask/images/test' --weights 'runs_mask/fine-tuning/weights/best.pt' --conf 0.6 --iou 0.45 --augment --project 'runs_mask' --name 'detect_test'

detect: weights=['runs_mask/fine-tuning/weights/best.pt'], source=../datasets/mask/images/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=True, visualize=False, update=False, project=runs_mask, name=detect_test, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-325-g3e85863 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 280 layers, 12315904 parameters, 0 gradients, 16.1 GFLOPs
image 1/85 /content/datasets/mask/images/test/maksssksksss768.png: 384x640 6 with masks, Done. (0.050s)
image 2/85 /content/datasets/mask/images/test/maksssksksss769.png: 384x640 3 with masks, Done. (0.041s)
image 3/85 /content/datasets/mask/images/test/maksssksksss770.png: 384x640 2 without masks, 2 with masks, Done. (0.035s)
image 4/85

## Export model

- TensorRT model will be saved in `runs_mask/feature_extraction/weights/best.engine`. Requires the modules `nvidia-pyindex` and `nvidia-tensorrt`.
- ONNX model will be saved in `runs_mask/feature_extraction/weights/best.onnx`. Requires the modules `onnx` and `onnx-simplifier`.
- For a list of supported model formats, refer to `export.py`.

In [8]:
!python export.py --weights 'runs_mask/feature_extraction/weights/best.pt' --include engine onnx --data 'data/mask_data.yaml' --device 0 --imgsz 640 640

export: data=data/mask_data.yaml, weights=['runs_mask/feature_extraction/weights/best.pt'], imgsz=[640, 640], batch_size=1, device=0, half=False, inplace=False, train=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['engine', 'onnx']
YOLOv5 🚀 v6.1-325-g3e85863 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 280 layers, 12315904 parameters, 0 gradients, 16.1 GFLOPs

PyTorch: starting from runs_mask/feature_extraction/weights/best.pt with output shape (1, 25500, 8) (24.0 MB)
requirements: nvidia-tensorrt not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.ngc.nvidia.com, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for nvidia-cublas-cu11: filename=nvidia_cublas_cu11-2022.4.8-py3-none-any.whl size=15624

## Inference Using Saved Model

- Using the model named `best.onnx` saved in `model` folder in the root directory. The model location can be changed using the `--weights` argument.
- Results are saved in `yolov5/runs_mask/detect_test` folder.
- Labels and bounding box coordinates (in x,y,w,h format) are saved in a .txt file in the `labels` folder stored within the previous folder. Can be toggled on/off using the `--save-txt` argument.

In [ ]:
!python detect.py --source ../datasets/mask/images/test --weights ../model/best.onnx --conf 0.6 --iou 0.45 --augment --save-txt --project runs_mask --name detect_test

detect: weights=['../model/best.onnx'], source=../datasets/mask/images/test, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=True, visualize=False, update=False, project=runs_mask, name=detect_test, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
fatal: cannot change to 'D:\Everything\Coding': No such file or directory
YOLOv5  2022-7-29 Python-3.8.6 torch-1.11.0+cu113 CUDA:0 (GeForce GTX 1060, 6144MiB)

Loading ..\model\best.onnx for ONNX Runtime inference...
image 1/85 D:\Everything\Coding Projects\face-mask-detection\datasets\mask\images\test\maksssksksss768.png: 640x640 7 with masks, Done. (0.030s)
image 2/85 D:\Everything\Coding Projects\face-mask-detection\datasets\mask\images\test\maksssksksss769.png: 640x640 3 with masks, Done. (0.026s)
image 3/85 D:\Everything\Coding 